# Fraud Detection

## Data loading

In [161]:
import pandas as pd
import numpy as np

In [162]:
df = pd.read_csv('../dataset/fraud_detection.csv')
df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,1744506,2018-09-30 00:00:01,4423,7637,12.29,15724801,182,0,0
1,1744507,2018-09-30 00:00:27,3680,5369,222.32,15724827,182,1,1
2,1744508,2018-09-30 00:00:38,3255,1246,8.79,15724838,182,0,0
3,1744509,2018-09-30 00:01:49,1353,8423,33.77,15724909,182,0,0
4,1744510,2018-09-30 00:02:09,1231,1382,60.86,15724929,182,0,0


In [163]:
df.shape

(9649, 9)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TRANSACTION_ID     9649 non-null   int64  
 1   TX_DATETIME        9649 non-null   object 
 2   CUSTOMER_ID        9649 non-null   int64  
 3   TERMINAL_ID        9649 non-null   int64  
 4   TX_AMOUNT          9649 non-null   float64
 5   TX_TIME_SECONDS    9649 non-null   int64  
 6   TX_TIME_DAYS       9649 non-null   int64  
 7   TX_FRAUD           9649 non-null   int64  
 8   TX_FRAUD_SCENARIO  9649 non-null   int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 678.6+ KB


In [165]:
df.describe()

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
count,9.649000e+03,9649.000000,9649.000000,9649.000000,9.649000e+03,9649.0,9649.000000,9649.000000
mean,1.749330e+06,2488.178982,4992.228003,53.479106,1.576829e+07,182.0,0.008291,0.018862
std,2.785571e+03,1448.295221,2880.338647,42.517344,1.819103e+04,0.0,0.090681,0.212719
min,1.744506e+06,0.000000,0.000000,0.000000,1.572480e+07,182.0,0.000000,0.000000
25%,1.746918e+06,1235.000000,2536.000000,20.540000,1.575521e+07,182.0,0.000000,0.000000
50%,1.749330e+06,2478.000000,4953.000000,44.180000,1.576846e+07,182.0,0.000000,0.000000
75%,1.751742e+06,3746.000000,7484.000000,76.660000,1.578122e+07,182.0,0.000000,0.000000
max,1.754154e+06,4999.000000,9999.000000,699.300000,1.581120e+07,182.0,1.000000,3.000000


In [166]:
df['TX_DATETIME'].value_counts()

TX_DATETIME
2018-09-30 10:44:46    4
2018-09-30 08:44:56    3
2018-09-30 08:39:25    3
2018-09-30 11:59:47    3
2018-09-30 11:02:41    3
                      ..
2018-09-30 09:45:19    1
2018-09-30 09:45:16    1
2018-09-30 09:45:14    1
2018-09-30 09:45:11    1
2018-09-30 23:59:57    1
Name: count, Length: 9016, dtype: int64

In [167]:
df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD',
       'TX_FRAUD_SCENARIO'],
      dtype='object')

import plotly.express as px

fig = px.line(df, x='TX_DATETIME', y='TX_AMOUNT', color='TX_FRAUD', markers=True)
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df, x='TX_DATETIME', y='TX_AMOUNT', color='TX_FRAUD_SCENARIO', markers=True)
fig.show()

In [175]:
X = df[['TX_AMOUNT']]
y = df['TX_FRAUD']

In [176]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X, y,
                                                       stratify=y,
                                                       random_state=100,
                                                       test_size=0.2)

X_train.shape, X_test.shape

((7719, 1), (1930, 1))

In [177]:
from sklearn.ensemble import IsolationForest

anomalyclassifier = IsolationForest(n_estimators=10)
anomalyclassifier.fit(X_train)

IsolationForest(n_estimators=10)

In [178]:
X_train['score'] = -anomalyclassifier.score_samples(X_train)
X_train.head() 

,TX_AMOUNT,score
4994,82.07,0.521481
9078,62.08,0.447080
7619,92.76,0.522045
9309,14.16,0.444215
8229,43.80,0.459297


In [ ]:
import plotly.express as px

fig = px.histogram(X_train['score'])
fig.show()

Threshold를 0.65으로 정해서 Anomaly를 구해보자

In [181]:
X_test['score'] = -anomalyclassifier.score_samples(X_test)
X_test.head() 

,TX_AMOUNT,score
7216,105.09,0.535843
5865,26.47,0.462165
3698,94.82,0.498355
4892,71.62,0.436910
6363,61.10,0.438537


In [183]:
X_test['Prediction'] = np.where(X_test['score'] > 0.65, 1, 0)
X_test

,TX_AMOUNT,score,Prediction
7216,105.09,0.535843,0
5865,26.47,0.462165,0
3698,94.82,0.498355,0
4892,71.62,0.436910,0
6363,61.10,0.438537,0
...,...,...,...
8049,30.40,0.453904,0
8587,1.94,0.549982,0
4045,59.72,0.451344,0
2915,8.89,0.489017,0


In [184]:
X_test['TX_FRAUD'] = y_test
X_test

,TX_AMOUNT,score,Prediction,TX_FRAUD
7216,105.09,0.535843,0,0
5865,26.47,0.462165,0,0
3698,94.82,0.498355,0,0
4892,71.62,0.436910,0,0
6363,61.10,0.438537,0,0
...,...,...,...,...
8049,30.40,0.453904,0,0
8587,1.94,0.549982,0,0
4045,59.72,0.451344,0,0
2915,8.89,0.489017,0,0


In [187]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(X_test['Prediction'], y_test)

array([[1897,   12],
       [  17,    4]])

Autoencoders are part of the large deep learning models family. Their goal is to learn representations to reconstruct descriptive variables, so they have been widely used for unsupervised learning problems. Anomaly detection, and in particular fraud detection, can be tackled with unsupervised or semi-supervised techniques. 

In this section, we used the autoencoder, and in particular its reconstruction error, as an indicator for fraud risk. Used solely (unsupervised method), it detects data points that are away from the rest of the distribution, which allows detecting many frauds but also introduces a lot of false alerts (e.g. genuine transactions that have rare characteristics). Therefore, it obtains a decent AUC ROC but low precision-based metrics. Used as an extra variable in a supervised method (semi-supervised usage), it can allow boosting the performance in specific settings.

In [188]:
print(classification_report(X_test['Prediction'], y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1909
           1       0.25      0.19      0.22        21

    accuracy                           0.98      1930
   macro avg       0.62      0.59      0.60      1930
weighted avg       0.98      0.98      0.98      1930



In [ ]:
import plotly.express as px


fig1 = px.histogram(
    X_test,
    x="TX_AMOUNT",
    color="Prediction",
    marginal="box",
    width=800,
    height=600,
)
fig1.show()



fig2 = px.histogram(
    X_test,
    x="TX_AMOUNT",
    color="TX_FRAUD",
    marginal="box",
    width=800,
    height=600,
)
fig.show()